# <center> Exploring Pre-WWII Soviet Socialist Realism with Text Analysis

<img src="image.png">

About: This notebook uses logistic regression and topic modeling to explore the particularities of the Soviet literary genre of socialist realism in the pre-WII Soviet Union.  

The python modules nltk, gensim, pymystem, and pyLDAvis are used in this notebook.

In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords 
import string
import gensim
import scipy
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from gensim import corpora
from gensim import models
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
from pymystem3 import Mystem
import pyLDAvis
import re

# Section 1 : Predictive Analysis

This section splits the corpus (both socialist realism and banned literature) into training and testing data and uses logistic regression to predict the values of the testing data.   

Load .csv with both socialist realism and banned literature. 

In [2]:
df = pd.read_csv("csv_files/data_for_log_reg_smaller_chunks.csv")
df = shuffle(df)

y = df['bannedorsoclit']


Clean and lemmatize the text.

In [3]:
stop = list(stopwords.words('russian'))
stop.extend(['это', 'свой', 'то', ' ', 'то', 'который'])
mystem = Mystem() 
exclude = list(string.punctuation)
exclude.extend

def clean(text):
    tokens = text.split(" ")
    tokens = [i.strip().lower() for i in tokens]
    tokens = [i for i in tokens if i not in exclude]
    tokens = [re.sub(r'[!|,|?|.|:|;]|\.{1,}$\)', "", item) for item in tokens]
    tokens = [i for i in tokens if i not in stop]
    cleanedtext = " ".join(tokens)
    lemmatized = mystem.lemmatize(cleanedtext.lower())
    lemmatized = [i for i in lemmatized if i not in exclude]
    lemmatized = [i for i in lemmatized if i not in stop]
    lemmatized = " ".join(str(x) for x in lemmatized)
    return lemmatized


df["cleanedtext"] = df["text"].apply(clean)

df.head()


,source,title,date,text,bannedorsoclit,cleanedtext
222,\n\nhttp://www.lib.ru/PROZA/SHOLOHOW/tihijdon1...,\nТихий Дон\n,4,"в рубке , постарше - отвиливали от занятий . Л...",1,рубка старший отвиливать занятие человек хрипн...
187,\n\nhttp://lib.ru/RUSSLIT/OSTROWSKIJ/rozhdenn_...,\n\nРожденные бурей\n\n,4,"даже тогда были далеки от Черного моря ) , и п...",1,далекий черный море погибать оттого каждый уез...
197,\n\nhttp://lib.ru/RUSSLIT/OSTROWSKIJ/kak_zakal...,\n\nКак закалялась сталь\n\n,5,это время со стороны мельницы в город въезжал ...,1,время сторона мельница город въезжать вооружен...
135,\n\nhttp://librebook.me/cement\n\n,\nЦемент\n,2,"проклятых… Вишь , морды какие нахолили ! .. Мо...",1,проклятый … вишь морда нахолить чертол коза...
136,\n\nhttp://librebook.me/cement\n\n,\nЦемент\n,3,"Здравствуйте , товарищ Чумалова ! Заведующий с...",1,здравствовать товарищ чумалов заведующий прийт...


Vectorize the text and split it into training and testing data

In [4]:
count_vectorizer = CountVectorizer(stop_words=None)
# count_vectorizer = TfidfVectorizer()
features = count_vectorizer.fit_transform(
    df['text'])

features_nd = features.toarray()

X_train, X_test, y_train, y_test = train_test_split(
                                             features_nd[0:len(df['text'])], y, 
                                             test_size=0.4, 
                                             random_state=53)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(163, 120538) (110, 120538) (163,) (110,)


Apply logistic regression.

In [5]:
log_model = LogisticRegression()
log_model = log_model.fit(X=X_train, y=y_train)
y_pred = log_model.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(log_model.score(X_test, y_test)))

kfold = model_selection.KFold(n_splits=10, random_state=7)
modelCV = LogisticRegression()
scoring = 'accuracy'
results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

confusion_matrix = confusion_matrix(y_test, y_pred)
print('confusion_matrix', confusion_matrix)

print(classification_report(y_test, y_pred))




Accuracy of logistic regression classifier on test set: 0.95
10-fold cross validation average accuracy: 0.951
confusion_matrix [[30  6]
 [ 0 74]]
             precision    recall  f1-score   support

          0       1.00      0.83      0.91        36
          1       0.93      1.00      0.96        74

avg / total       0.95      0.95      0.94       110



Results indicate that, logistic regression, with a certain degree of accuracy can differentiate between the socialist realism and banned literature corpora.  It is likely that accuracy might be even higher, were deep learning techniques to be applied. 

# Section 2: Topic Modeling

Import .csv with data from entire socialist realist literature corpus. 

In [6]:
litdata = pd.read_csv("csv_files/socialist_realism_for_tm.csv")
litdata['text'].head(5)

0     СОДЕРЖАНИЕ I. Морозка II . Мечик III . Шестое...
1     . Многие открыто роптали на `` подлость и нес...
2     бровей на тоненькую девушку с длинными косами...
3     время люди очнулись и поняли , что наступило ...
4     кричал Рябец , навалившись на одно слово и вс...
Name: text, dtype: object

Define stopwords, punctuation, read in output from named entity recognition (NER) script.

In [7]:
with open('NERoutput.txt') as fp:
    lines = fp.read().splitlines()

lines = [i.lower() for i in lines]

stop = list(stopwords.words('russian'))
stop.extend(['это', 'свой', 'то', ' ','который'])
stop.extend(lines)

punctuation = list(string.punctuation)
punctuation.extend(['--', '—', '«', '»'," -- ", "\n", "\r", "\t","-" ,"-", "...", "…", " - ", " « ", " ", "..", "``", "\"\"","\'\'"])


Use function to create a new column in pandas df with cleaned text.

In [8]:
# Create new column in dataframe that is literary texts without stopwords and punctuation, as well as without
# named entities for people, as identified in NERoutput.txt, which was created with NERscript.py

def cleantext(text):
    tokens = text.split(" ")
    tokenslower = [i.strip().lower() for i in tokens]
#the following list comprehension gets rid of word-final punctuation, while retaining word-medial punctuation in words like "кто-то"
    tokens_no_word_final_punc = [re.sub(r'[!|,|?|.|:|;]|\.{1,}$\)', "", item) for item in tokenslower]
    tokens_no_stop_or_punc = [i for i in tokens_no_word_final_punc if i not in stop if i not in punctuation]
#the following line gets rid of blank, empty spaces in each list    
    tokens_no_stop_or_punc = list(filter(None, tokens_no_stop_or_punc))
    return tokens_no_stop_or_punc
    
litdata["no_punc_or_stop"] = litdata["text"].apply(cleantext)
no_punc_or_stop = litdata["no_punc_or_stop"]

# Print first five rows of new column

no_punc_or_stop.head(5)

0    [содержание, i, ii, iii, шестое, чувство, iv, ...
1    [многие, открыто, роптали, подлость, несознани...
2    [бровей, тоненькую, девушку, длинными, косами,...
3    [время, люди, очнулись, поняли, наступило, утр...
4    [кричал, рябец, навалившись, одно, слово, веря...
Name: no_punc_or_stop, dtype: object

Use pymystem to lemmatize the text. 

In [9]:
mystem = Mystem() 


def lemmatize(text):
    joinedtokens = " ".join(text)
    lemmatized = mystem.lemmatize(joinedtokens)
    lemmatized = [i for i in lemmatized if i not in stop if i not in punctuation]
    return lemmatized
    
litdata["lemmatized"] = litdata["no_punc_or_stop"].apply(lemmatize)

litdata["lemmatized"].head(5)

0    [содержание, i, ii, iii, шестой, чувство, iv, ...
1    [многие, открыто, роптать, подлость, несознани...
2    [бровь, тоненький, девушка, длинный, коса, под...
3    [время, человек, очнуться, понимать, наступать...
4    [кричать, рябец, наваливаться, слово, верить, ...
Name: lemmatized, dtype: object

Topic modeling with gensim, using Latent Semantic Indexing (LSI)

In [10]:
dictionary = corpora.Dictionary(litdata["lemmatized"] )
doc_term_matrix = [dictionary.doc2bow(item) for item in litdata["lemmatized"] ]
lsi = models.LsiModel(doc_term_matrix, num_topics=4, id2word=dictionary)
output = lsi.print_topics(num_topics=4, num_words = 4)
print (output)

[(0, '0.286*"сказать" + 0.255*"человек" + 0.228*"рука" + 0.204*"говорить"'), (1, '0.324*"мать" + 0.322*"сказать" + -0.246*"казак" + 0.227*"человек"'), (2, '-0.202*"знать" + -0.183*"дело" + 0.177*"глаз" + 0.168*"голова"'), (3, '-0.286*"казак" + 0.228*"человек" + -0.223*"мать" + -0.157*"говорить"')]


Results are skewed because of the length of certain works in the corpus, primarily "Quiet Flows the Don".  Next step, try to rectify these results by using a smaller, more even amount of text from each work in the corpus.

## What are the results with smaller chunked files?

In [11]:
import pandas as pd

newsocreal = pd.read_csv("csv_files/socreal_files_chunked_smaller.csv")
newsocreal.head(5)

,title,name,date,english metadata,source,text
0,Разгром,"Фадеев , Александр Александрович",1926,"Aleksandr Fadeev , The Rout , 1926",http : //lib.ru/RUSSLIT/FADEEW/razgrom.txt,СОДЕРЖАНИЕ I. Морозка II . Мечик III . Шестое...
1,\n\nРазгром\n\n,"Фадеев, Александр Александрович\n\n",2,"\n\nAleksandr Fadeev, The Rout, 1926\n\n",\n\nhttp://lib.ru/RUSSLIT/FADEEW/razgrom.txt\n\n,"пыльным , сумасшедшим карьером . -- Обожди-и ,..."
2,\n\nРазгром\n\n,"Фадеев, Александр Александрович\n\n",3,"\n\nAleksandr Fadeev, The Rout, 1926\n\n",\n\nhttp://lib.ru/RUSSLIT/FADEEW/razgrom.txt\n\n,.. -- снова повторил Дубов . -- Блуди ! Посмот...
3,\n\nРазгром\n\n,"Фадеев, Александр Александрович\n\n",4,"\n\nAleksandr Fadeev, The Rout, 1926\n\n",\n\nhttp://lib.ru/RUSSLIT/FADEEW/razgrom.txt\n\n,"Ведь надо как-то , умеют же другие ... '' В мы..."
4,\n\nРазгром\n\n,"Фадеев, Александр Александрович\n\n",5,"\n\nAleksandr Fadeev, The Rout, 1926\n\n",\n\nhttp://lib.ru/RUSSLIT/FADEEW/razgrom.txt\n\n,"ребят , вернувшихся из-за реки . Дубов узнал ,..."


Apply all of the same steps to the .csv with smaller chunks of each individual work in the corpus.  

In [12]:
with open('NERoutput.txt') as fp:
    lines = fp.read().splitlines()

lines = [i.lower() for i in lines]
lines = " ".join(lines)
mystem = Mystem() 
lemmatizedlines = mystem.lemmatize(lines)
lemmatizedlines = [i for i in lemmatizedlines if i != " "]

stop = list(stopwords.words('russian'))
stop.extend(['это', 'свой', 'то', ' ','который', 'говорить', 'сказать', 'знать', 'спрашивать', 'вопрос'])
stop.extend(lemmatizedlines)

punctuation = list(string.punctuation)
punctuation.extend(['--', '—', " – ", ' —', '«', '»'," -- ", " – – ", "\n", "\r", "\t","-" ,"-", "...", "…", " - ", " « ", " ", "..", "``", "\"\"","\'\'"])

def cleantext(text):
    tokens = text.split(" ")
    tokenslower = [i.strip().lower() for i in tokens]
#the following list comprehension gets rid of word-final punctuation, while retaining word-medial punctuation in words like "кто-то"
    tokens_no_word_final_punc = [re.sub(r'[!|,|?|.|:|;]|\.{1,}$\)', "", item) for item in tokenslower]
    tokens_no_stop_or_punc = [i for i in tokens_no_word_final_punc if i not in stop if i not in punctuation]
#the following line gets rid of blank, empty spaces in each list    
    tokens_no_stop_or_punc = list(filter(None, tokens_no_stop_or_punc))
    return tokens_no_stop_or_punc

newsocreal["no_punc_or_stop"] = newsocreal["text"].apply(cleantext)
cleaned = newsocreal["no_punc_or_stop"]

mystem = Mystem() 

def lemmatize(text):
    joinedtokens = " ".join(text)
    lemmatized = mystem.lemmatize(joinedtokens)
    lemmatized = [i for i in lemmatized if i not in stop if i not in punctuation]
    return lemmatized

newsocreal["lemmatized"] = newsocreal["no_punc_or_stop"].apply(lemmatize)

#Topic Modeling with Gensim, using Latent Semantic Indexing
dictionary = corpora.Dictionary(newsocreal["lemmatized"] )
doc_term_matrix = [dictionary.doc2bow(item) for item in newsocreal["lemmatized"] ]
lsi = models.LsiModel(doc_term_matrix, num_topics=4, id2word=dictionary)
output = lsi.print_topics(num_topics=4, num_words = 4)

print (output)




[(0, '-0.268*"человек" + -0.246*"рука" + -0.218*"глаз" + -0.158*"голова"'), (1, '0.416*"мать" + 0.205*"человек" + -0.173*"крепость" + -0.165*"дело"'), (2, '0.177*"крепость" + 0.167*"дело" + -0.158*"голова" + -0.155*"казак"'), (3, '0.288*"товарищ" + 0.165*"завод" + 0.163*"рабочий" + -0.142*"двор"')]


What happens with Latent Dirichlet Allocation (LDA) as opposed to LSI?

In [13]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

ldamodel = gensim.models.ldamodel.LdaModel(doc_term_matrix, num_topics=3, id2word = dictionary, passes=20)
ldaoutput = ldamodel.print_topics(num_topics=3, num_words=4)
print (ldaoutput)

[(0, '0.005*"глаз" + 0.004*"рука" + 0.004*"голова" + 0.004*"становиться"'), (1, '0.006*"товарищ" + 0.005*"рука" + 0.004*"глаз" + 0.003*"человек"'), (2, '0.007*"человек" + 0.005*"рука" + 0.005*"глаз" + 0.004*"лицо"')]


In [14]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      66.262472        1       1  0.035297  0.063377
0      19.871130        1       2 -0.089034 -0.008186
1      13.866398        1       3  0.053737 -0.055192, topic_info=      Category         Freq         Term        Total  loglift  logprob
term                                                                   
1368   Default   859.000000      товарищ   859.000000  30.0000  30.0000
1167   Default  1967.000000         рука  1967.000000  29.0000  29.0000
224    Default  1737.000000         глаз  1737.000000  28.0000  28.0000
3012   Default   460.000000      рабочий   460.000000  27.0000  27.0000
26     Default   387.000000         баба   387.000000  26.0000  26.0000
6010   Default   278.000000        завод   278.000000  25.0000  25.0000
5609   Default   367.000000        казак   367.000000  24.0000  24.0000
232    Default  1273.000000       голова  1273.000000  23.0000  23.0000
426    Default  1221.000000         идти  1221.000000  22.0000  22.0000
575    Default   385.000000       лошадь   385.000000  21.0000  21.0000
736    Default   767.000000         нога   767.000000  20.0000  20.0000
3942   Default   264.000000         гора   264.000000  19.0000  19.0000
1293   Default  1244.000000  становиться  1244.000000  18.0000  18.0000
412    Default   569.000000        земля   569.000000  17.0000  17.0000
1310   Default   968.000000       стоять   968.000000  16.0000  16.0000
854    Default   570.000000        очень   570.000000  15.0000  15.0000
2444   Default   170.000000        хутор   170.000000  14.0000  14.0000
254    Default   725.000000       давать   725.000000  13.0000  13.0000
269    Default  1091.000000         дело  1091.000000  12.0000  12.0000
227    Default   610.000000      глядеть   610.000000  11.0000  11.0000
1255   Default   591.000000     смотреть   591.000000  10.0000  10.0000
175    Default   778.000000        время   778.000000   9.0000   9.0000
233    Default   768.000000        голос   768.000000   8.0000   8.0000
130    Default   816.000000       видеть   816.000000   7.0000   7.0000
1099   Default   372.000000       работа   372.000000   6.0000   6.0000
2097   Default   867.000000        пойти   867.000000   5.0000   5.0000
567    Default  1212.000000         лицо  1212.000000   4.0000   4.0000
743    Default   576.000000         ночь   576.000000   3.0000   3.0000
3768   Default   287.000000       солдат   287.000000   2.0000   2.0000
6200   Default   126.000000      повозка   126.000000   1.0000   1.0000
...        ...          ...          ...          ...      ...      ...
10071   Topic3    32.729170       жидкий    51.156278   1.5291  -7.3873
1167    Topic3   274.405038         рука  1967.181058   0.0060  -5.2609
224     Topic3   219.615703         глаз  1737.210183  -0.0925  -5.4836
1099    Topic3    87.538495       работа   372.861135   0.5266  -6.4034
426     Topic3   158.733456         идти  1221.226089  -0.0647  -5.8083
2197    Topic3    62.200250         пыль   195.607644   0.8299  -6.7452
1255    Topic3   104.892056     смотреть   591.844401   0.2454  -6.2226
58      Topic3    73.264561         бить   283.041715   0.6242  -6.5814
269     Topic3   137.950742         дело  1091.134212  -0.0924  -5.9486
1310    Topic3   129.040366       стоять   968.310570  -0.0397  -6.0154
130     Topic3   117.503779       видеть   816.894228   0.0367  -6.1091
175     Topic3   114.768143        время   778.651907   0.0611  -6.1326
254     Topic3   108.352972       давать   725.762801   0.0739  -6.1901
232     Topic3   139.224816       голова  1273.006449  -0.2373  -5.9394
1484    Topic3   168.576038      человек  2146.718217  -0.5686  -5.7481
567     Topic3   130.495823         лицо  1212.597357  -0.2535  -6.0042
302     Topic3    88.530983         друг   512.073401   0.2206  -6.3922
854     Topic3    86.876656        очень   570.378729   0.0939  -6.4110
2028    Topic3 

## Now, let's examine the banned literature corpus. 

In [15]:
bannedlit = pd.read_csv("csv_files/banned_lit_for_tm_smaller_chunks.csv")
bannedlit.head(5)

,title,name,date,english metadata,source,text
0,Собачье сердце,"Булгаков , Михаил",1925,"Mikhail Bulgakov , A Dog ’ s Heart , 1924",http : //bulgakov.lit-info.ru/bulgakov/proza/...,"У-у-у-у-у-гу-гуг-гуу ! О , гляньте на меня , ..."
1,\n\nСобачье сердце\n\n,"Булгаков, Михаил\n",2,"\n\nMikhail Bulgakov, A Dog’s Heart, 1924\n\n\n",\n\nhttp://bulgakov.lit-info.ru/bulgakov/proza...,", дорогой профессор , только в виде опыта . - ..."
2,\n\nСобачье сердце\n\n,"Булгаков, Михаил\n",3,"\n\nMikhail Bulgakov, A Dog’s Heart, 1924\n\n\n",\n\nhttp://bulgakov.lit-info.ru/bulgakov/proza...,Ястребиные ноздри его раздувались . Набравшись...
3,\n\nСобачье сердце\n\n,"Булгаков, Михаил\n",4,"\n\nMikhail Bulgakov, A Dog’s Heart, 1924\n\n\n",\n\nhttp://bulgakov.lit-info.ru/bulgakov/proza...,правом боку следы совершенно зажившего ожога ....
4,\n\nСобачье сердце\n\n,"Булгаков, Михаил\n",5,"\n\nMikhail Bulgakov, A Dog’s Heart, 1924\n\n\n",\n\nhttp://bulgakov.lit-info.ru/bulgakov/proza...,трубку в рогульки . Голубая радость разлилась ...


In [19]:
with open('NERoutputBanned.txt') as fp:
    lines = fp.read().splitlines()

lines = [i.lower() for i in lines]
joinedlines = " ".join(lines)
mystem = Mystem() 
lemmatizedlines = mystem.lemmatize(joinedlines)
lemmatizedlines = [i for i in lemmatizedlines if i != " "]

stop = list(stopwords.words('russian'))
stop.extend(['это', 'свой', 'то', ' ','который', 'говорить', 'сказать', 'знать', 'спрашивать', 'вопрос','наш'])
stop.extend(lemmatizedlines)

punctuation = list(string.punctuation)
punctuation.extend(['--', '—', " – ", ' —', '«', '»'," -- ", " – – ", "\n", "\r", "\t","-" ,"-", "...", "…", " - ", " « ", " ", "..", "``", "\"\"","\'\'"])

def cleantext(text):
    tokens = text.split(" ")
    tokenslower = [i.strip().lower() for i in tokens]
#the following list comprehension gets rid of word-final punctuation, while retaining word-medial punctuation in words like "кто-то"
    tokens_no_word_final_punc = [re.sub(r'[!|,|?|.|:|;]|\.{1,}$\)', "", item) for item in tokenslower]
    tokens_no_stop_or_punc = [i for i in tokens_no_word_final_punc if i not in stop if i not in punctuation]
#the following line gets rid of blank, empty spaces in each list    
    tokens_no_stop_or_punc = list(filter(None, tokens_no_stop_or_punc))
    return tokens_no_stop_or_punc

bannedlit["no_punc_or_stop"] = bannedlit["text"].apply(cleantext)
cleaned = bannedlit["no_punc_or_stop"]

mystem = Mystem() 

def lemmatize(text):
    joinedtokens = " ".join(text)
    lemmatized = mystem.lemmatize(joinedtokens)
    lemmatized = [i for i in lemmatized if i not in stop if i not in punctuation]
    return lemmatized

bannedlit["lemmatized"] = bannedlit["no_punc_or_stop"].apply(lemmatize)

#Topic Modeling with Gensim, using Latent Semantic Indexing
dictionaryb = corpora.Dictionary(bannedlit["lemmatized"] )
doc_term_matrixb = [dictionaryb.doc2bow(item) for item in bannedlit["lemmatized"] ]
lsi = models.LsiModel(doc_term_matrixb, num_topics=4, id2word=dictionaryb)
output = lsi.print_topics(num_topics=4, num_words = 4)

print (output)


[(0, '0.333*"человек" + 0.184*"рука" + 0.177*"глаз" + 0.155*"жизнь"'), (1, '-0.269*"человек" + 0.227*"профессор" + 0.178*"глаз" + -0.159*"жизнь"'), (2, '-0.359*"профессор" + -0.214*"отвечать" + -0.213*"человек" + -0.138*"кабинет"'), (3, '-0.203*"год" + -0.192*"человек" + -0.143*"отец" + -0.136*"дом"')]


Now, what happens with LDA?

In [22]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

ldamodel = gensim.models.ldamodel.LdaModel(doc_term_matrixb, num_topics=4, id2word = dictionaryb, passes=20)
ldaoutput = ldamodel.print_topics(num_topics=3, num_words=4)
print (ldaoutput)

[(3, '0.007*"человек" + 0.004*"жизнь" + 0.004*"рука" + 0.004*"весь"'), (1, '0.007*"человек" + 0.004*"рука" + 0.004*"жизнь" + 0.003*"год"'), (2, '0.007*"человек" + 0.004*"рука" + 0.003*"глаз" + 0.003*"идти"')]


In [23]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, doc_term_matrixb, dictionaryb)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      34.207917        1       1  0.042869  0.021806
3      29.833048        1       2  0.081240 -0.017346
0      28.064041        1       3 -0.063430 -0.074098
2       7.894994        1       4 -0.060680  0.069638, topic_info=      Category         Freq         Term        Total  loglift  logprob
term                                                                   
1010   Default   299.000000    профессор   299.000000  30.0000  30.0000
1378   Default  1354.000000      человек  1354.000000  29.0000  29.0000
1087   Default   803.000000         рука   803.000000  28.0000  28.0000
1203   Default   261.000000         стол   261.000000  27.0000  27.0000
19469  Default    95.000000   прушевский    95.000000  26.0000  26.0000
19695  Default    95.000000     активист    95.000000  25.0000  25.0000
211    Default   519.000000          год   519.000000  24.0000  24.0000
413    Default   564.000000         идти   564.000000  23.0000  23.0000
449    Default   143.000000      кабинет   143.000000  22.0000  22.0000
8461   Default    94.000000            i    94.000000  21.0000  21.0000
1028   Default   201.000000       работа   201.000000  20.0000  20.0000
503    Default    79.000000          кот    79.000000  19.0000  19.0000
16498  Default    86.000000     сафронов    86.000000  18.0000  18.0000
2750   Default   233.000000         мать   233.000000  17.0000  17.0000
757    Default   202.000000         окно   202.000000  16.0000  16.0000
3995   Default    67.000000     командир    67.000000  15.0000  15.0000
11270  Default   105.000000       корова   105.000000  14.0000  14.0000
18419  Default    99.000000  федератовна    99.000000  13.0000  13.0000
2891   Default   143.000000        война   143.000000  12.0000  12.0000
6682   Default    74.000000        степа    74.000000  11.0000  11.0000
223    Default   129.000000         гора   129.000000  10.0000  10.0000
13079  Default    90.000000        балка    90.000000   9.0000   9.0000
13450  Default    36.000000     березкин    36.000000   8.0000   8.0000
830    Default   264.000000       первый   264.000000   7.0000   7.0000
24     Default   126.000000       бежать   126.000000   6.0000   6.0000
14801  Default    30.000000    лозовский    30.000000   5.0000   5.0000
8978   Default   175.000000        мужик   175.000000   4.0000   4.0000
4031   Default   104.000000    секретарь   104.000000   3.0000   3.0000
1195   Default   636.000000  становиться   636.000000   2.0000   2.0000
19618  Default    63.000000       елисей    63.000000   1.0000   1.0000
...        ...          ...          ...          ...      ...      ...
10284   Topic4     8.870983         рота    15.949834   1.9523  -7.5101
1378    Topic4   106.445624      человек  1354.392052  -0.0045  -5.0253
1203    Topic4    38.974569         стол   261.122058   0.6369  -6.0300
2891    Topic4    27.357086        война   143.655431   0.8805  -6.3839
1028    Topic4    32.389370       работа   201.763225   0.7097  -6.2151
1087    Topic4    64.947137         рука   803.954661   0.0230  -5.5193
757     Topic4    30.863895         окно   202.122089   0.6597  -6.2633
24      Topic4    24.451269       бежать   126.070732   0.8988  -6.4962
211     Topic4    47.376344          год   519.206169   0.1448  -5.8348
413     Topic4    48.063930         идти   564.371357   0.0758  -5.8204
2750    Topic4    31.485386         мать   233.012537   0.5374  -6.2434
1195    Topic4    45.413160  становиться   636.119486  -0.1006  -5.8771
204     Topic4    48.481123         глаз   789.884583  -0.2518  -5.8117
278     Topic4    33.460674          дом   379.768197   0.1098  -6.1825
267     Topic4    32.506735         дело   376.140710   0.0904  -6.2114
3514    Topic4    22.537733         друг   145.893950   0.6713  -6.5777
830     Topic4    27.768008       первый   264.950353   0.2833  -6.3690
2054    Topic4    24.373612      